# Get Vasari letters from Fondazione Memofonte (https://www.memofonte.it/ricerche/giorgio-vasari/) with Beautiful Soup

#### based on Jeri Wieringa, "Intro to Beautiful Soup," The Programming Historian 1 (2012), https://doi.org/10.46430/phen0008.

In [1]:
import urllib.request, urllib.error, urllib.parse
import csv
from bs4 import BeautifulSoup
import html2text
import os
from datetime import datetime
import numpy as np
import pandas as pd

In [2]:
# function to check if 'nan'

def isnan(x):
    return x != x
    

In [3]:
data_path = 'data/results/letters/'

## Iterate over all URLs and save as .html

#### make list of URLs

In [2]:
# letter-id ranging from 1 to 1151

url_list = []
url_base = "https://www.memofonte.it/home/ricerca/singolo_17.php?id="
for i in range (1151):
    url_list.append(url_base+ str(i+1))
    
len(url_list)

1151

### save all as .html

In [40]:
count = 0
for i in url_list:
    count+=1
    response = urllib.request.urlopen(i)
    webContent = response.read()
    #filename = 'data/results/letters/html/letter-'+str(i)'.html'
    f = open('data/results/letters/html/letter-'+str(count)+'.html', 'wb')
    f.write(webContent)
    f.close()
    

## Open all .html, make Beautiful Soups, extract information, save all letters in a single CSV

In [3]:
list_of_htmls = os.listdir('data/results/letters/html')
len(list_of_htmls)
#list_of_htmls

1151

In [4]:
# test code and output for single file before iterating over 1151

csv_row = list()

soup = BeautifulSoup(open('data/results/letters/html/'+str(list_of_htmls[0])), "lxml")

p_array = soup.find_all('p')
csv_row.append(int(p_array[0].text[(len('Numero d\'ordine:')+1):]))
csv_row.append(((p_array[1].text[(len('Data:')+1):])))
csv_row.append(p_array[2].text[(len('Intestazione:')+1):])
csv_row.append(p_array[3].text[(len('Segnatura:')+1):])
csv_row.append(p_array[4].text[(len('Fonte:')+1):])
csv_row.append(p_array[5].text[(len('Bibliografia:')+1):])

csv_row

[1051,
 '26 12  1572',
 'VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI IN ROMA',
 'ASA, AV, 14 (XLVIII), cc. 146-148.',
 'Molto Magnifico messer Giorgio. Ho ricevuto la vostra con la distesa delle istorie della sala, le quali considererò, e perché mi ha dato noia un po’ di scesa et non ne son libero, indugierò a questa altra settimana a darne più piena risposta. Messer Vincenzo non credo che pensi più alla messa, ma quando si potessi confermare nel privileggio del legittimare et altri indulti che non toccono il concilio, credo sarebbe contento, che qua non lo ammettono, e però forse bisognerebbe questa confermazione; pure ci sarà tempo. De la cosa mia come mi dite ne starò a buona speranza; e se voi vedete che la camini bene, il meglio sarebbe stringnere e darli fine, che cosa fatta importa assai e si schifa molte cose, che col tempo possono occorrere, che si dice che l’indugio pigla vizio; ma se ci fusse qualche difficultà, allora è bene non strignere, che il far male ha da essere l’u

In [34]:
# inspect data
#soup.prettify

In [70]:
# test how to convert the date

datetime_object = datetime.strptime((p_array[1].text[(len('Data:')+1):]), "%d %m %Y")
datetime_object

datetime.datetime(1572, 12, 26, 0, 0)

### Make one CSV File for all letters

In [54]:
# make CSV File
# changed filename after first run from letters.csv to letters_2.csv



#csv_file = csv.writer(open('data/results/letters/letters_2.csv', "w"))

# Write column headers as the first line
#csv_file.writerow(["Numero d'ordine", "Data", "Intestazione", "Segnatura", "Fonte", "Bibliografia"]) 

64

### iterate over all .html files, save as csv

In [71]:
# open all letter.html, make soup, extract information, collect it in a list, append list as a row to csv_file

for i in list_of_htmls:
    csv_row = list() # make new row
    soup = BeautifulSoup(open('data/results/letters/html/'+i), "lxml") # make soup
    p_array = soup.find_all('p') # take the information one by one 
    csv_row.append(int(p_array[0].text[(len('Numero d\'ordine:')+1):])) # append each to the list
    csv_row.append(p_array[1].text[(len('Data:')+1):])
    csv_row.append(p_array[2].text[(len('Intestazione:')+1):])
    csv_row.append(p_array[3].text[(len('Segnatura:')+1):])
    csv_row.append(p_array[4].text[(len('Fonte:')+1):])
    csv_row.append(p_array[5].text[(len('Bibliografia:')+1):])
    #csv_file.writerow(csv_row)

#soup = BeautifulSoup(open('data/results/letters/vasari-letter1.html'), "lxml")
# extract all "p"
#p_array = soup.find_all('p')
#p_array[1].text

## open CSV-file as dataframe

In [7]:
filename = 'data/results/letters/letters.csv'
csv_file = csv.reader(open(filename, "r"))
dataframe = pd.read_csv(filename)
dataframe

,Numero d'ordine,Data,Intestazione,Segnatura,Fonte,Bibliografia
0,1051,26 12 1572,VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI ...,"ASA, AV, 14 (XLVIII), cc. 146-148.",Molto Magnifico messer Giorgio. Ho ricevuto la...,"Frey 1930, pp. 736-737."
1,791,24 12 1569,GIORGIO VASARI IN FIRENZE A ORDINE DI SANTO ST...,"ASPI, Carte Bonaini, IX.",Illustrissimi Signori miei observandissimi. Co...,"Frey 1930, pp. 476-477."
2,284,20 04 1559,GABRIEL FIAMMA IN PADOVA A GIORGIO VASARI IN F...,"ASA, AV, 13 (XLVII), cc. 53, 59.",Magnifico Signor mio osservandissimo. Ho scrit...,"Frey 1923, pp. 513-514."
3,1114,1 08 1573,GIORGIO VASARI IN FIRENZE AI DEPUTATI ALLE FAB...,"AAA, filza CC, Registro di lettere, i, 1569, I...","Molto magnifici Signori Deputati, Signori mia ...","Frey 1930, pp. 810-811."
4,904,3 09 1571,GUGLIELMO SANGALLETTI IN ROMA A GIORGIO VASARI...,"ASA, AV, 15 (XLIX), cc. 174, 177.",Molto Magnifico Signor Cavaliere. La presente ...,"Frey 1930, pp. 599-600."
...,...,...,...,...,...,...
1146,356,1 01 1562,GIORGIO VASARI IN FIRENZE A JACOPO GUIDI,"ASF, Carte Guidi, c. 356.","Reverendissimo monsignore patron mio,io mando ...","Palli D’Addario 1985, pp. 379-380 (post 2 giug..."
1147,585,18 06 1565,GIORGIO VASARI IN FIRENZE A VINCENZO BORGHINI...,"ASF, CdA, II, I, n. 30.",Reverendo Signor mio. S’è riceuto la vostra le...,"Gaye 1839-1840, III, p. 185; Milanesi 1878-18..."
1148,839,29 08 1570,VINCENZO BORGHINI IN POPPIANO A GIORGIO VASARI,"ASA, AV, 14 (XLVIII), c. 70.",Messer Giorgio mio. Io vi mando una lunga dice...,"Frey 1930, pp. 522-523."
1149,993,6 06 1572,BARTOLOMMEO BUSSOTTO IN ROMA A GIORGIO VASARI ...,"ASA, AV, 10 (XLIV), cc. 83, 86.",Molto magnifico Signor come fratello. Scrissi ...,"Frey 1930, pp. 681-682."


## get rid of nan-values

'nan' count in total: 11 rows with nan values 
--> replace with '0' and collect index for documentation (row-indices found: 17, 101, 268, 368, 491, 569, 617, 703, 740, 928, 985)

In [8]:
count = 0
for i in range(len(dataframe)): # check column by column if isnan and replace with '0'
    
    if isnan(dataframe['Intestazione'][i]): 
        print('Intestazione, ', count)
        (dataframe['Intestazione'][i]) = '0'
        
    if isnan(dataframe['Segnatura'][i]):
        #print('Segnatura', count)
        (dataframe['Segnatura'][i]) = '0'
        
    if isnan(dataframe['Fonte'][i]):
        #print('Fonte', count)
        (dataframe['Fonte'][i]) = '0'
        
    if isnan(dataframe['Bibliografia'][i]):
        #print('Bibliografia', count)
        (dataframe['Bibliografia'][i]) = '0'
    
    
    count+=1
            

Intestazione,  17
Intestazione,  101
Intestazione,  268
Intestazione,  368
Intestazione,  491
Intestazione,  569
Intestazione,  617
Intestazione,  703
Intestazione,  740
Intestazione,  928
Intestazione,  985


<ipython-input-8-f6e2de138291>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  (dataframe['Intestazione'][i]) = '0'
<ipython-input-8-f6e2de138291>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  (dataframe['Segnatura'][i]) = '0'
<ipython-input-8-f6e2de138291>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  (dataframe['Fonte'][i]) = '0'
<ipython-input-8-f6e2de138291>:18: SettingWithCopyWarning: 
A value is trying to be set on a cop

# Distinguish letters written by Vasari from himself copies of his nefew

example for signature of letter copied by nefew (called Giorgio Vasari the younger) 
BRF, 2354, cc. 12v-14, copia di Giorgio Vasari il Giovane.

number of letters filtered at http://vasariscrittore.memofonte.it/archivio?
390

In [34]:
Vasari = list() # list for letters sent by Vasari
Vasari_no_copies = list() # filter out the copies of his nefew
#Va_indices = list()



#    if dataframe['Intestazione'][i].startswith('GIORGIO VASARI') or dataframe['Intestazione'][i].startswith(' VINCENZO BORGHINI'): # VINCENZO BORGHINI
count = 0
for i in range(len(dataframe)):
    count +=1 
    # all letters Vasari sent
    if dataframe['Intestazione'][i].startswith('GIORGIO VASARI') or dataframe['Intestazione'][i].startswith(' GIORGIO VASARI'): # GIORGIO VASARI
        #print(dataframe['Intestazione'][i])
        #print(dataframe['Segnatura'][i], count)
        Vasari.append(list(dataframe.iloc[i]))
    # copies filtered out
    if (dataframe['Intestazione'][i].startswith('GIORGIO VASARI') or dataframe['Intestazione'][i].startswith(' GIORGIO VASARI')) and 'copia' not in dataframe['Segnatura'][i]: # filter out copies
        #print(dataframe['Segnatura'][i], count)
        Vasari_no_copies.append(list(dataframe.iloc[i]))

        
len(Vasari), len(Vasari_no_copies)

(404, 363)

In [35]:
# check
'copia' in dataframe['Segnatura'][29]

True

Make new dataframes and save as csv files

In [18]:
columns = ["Numero d'ordine", "Data", "Intestazione", "Segnatura", "Fonte", "Bibliografia"]
data_path = 'data/results/letters/'

In [36]:
df_Vasari = pd.DataFrame(Vasari, columns = columns)
df_Vasari_no_copies = pd.DataFrame(Vasari_no_copies, columns = columns)


In [37]:
df_Vasari.to_csv(data_path + 'Vasari-all.csv', index = False, header=True)
df_Vasari_no_copies.to_csv(data_path + 'Vasari-no_copies.csv', index = False, header=True)


## split dataframe for different involved authors / contributing authorship candidates

Cosimo Bartoli and Pierfrancesco Giambullari involved as Co-Authors after Gerd Blum (https://archiv.ub.uni-heidelberg.de/artdok/1948/1/Blum_Provvidenza_e_progresso_2010.pdf) 

Pierfrancesco Giambullari, Cosimo Bartoli, Giovan Battista Gelli und Carlo Lenzoni involved in the preparation of the edition of 1550; massive contribution of Vincenzio Borghinis concerning the indices and growing involvement over the following decennies (Alessandro Nova: Vasari versus Vasari, S.151)

In [9]:
# get Pierfrancesco Giambullari, Cosimo Bartoli, Vincenzio Borghini
# examples of the signatures copied from http://vasariscrittore.memofonte.it/archivio# 
# COSIMO BARTOLI IN FIRENZE A GIORGIO VASARI IN ROMA (73 letters)
# VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI IN ROMA (131 letters)
# PIERFRANCESCO GIAMBULLARI IN FIRENZE A GIORGIO VASARI IN AREZZO (6 letters)

Borghini = list() # collect whole entries written by Vincenzo Borghini
Bo_indices = list() # collect indices of Borghini entries

Giambullari = list() # collect Pierfrancesco Giambullari
G_indices = list()

Bartoli = list() # collect Pierfrancesco Giambullari
Ba_indices = list()

for i in range(len(dataframe)):

    
    if dataframe['Intestazione'][i].startswith('VINCENZO BORGHINI') or dataframe['Intestazione'][i].startswith(' VINCENZO BORGHINI'): # VINCENZO BORGHINI
        Borghini.append(list(dataframe.iloc[i]))
    
    if 'COSIMO BARTOLI' in dataframe['Intestazione'][i]: # COSIMO BARTOLI
        Bartoli.append(list(dataframe.iloc[i]))
    
    if 'PIERFRANCESCO GIAMBULLARI' in dataframe['Intestazione'][i]:
        Giambullari.append(list(dataframe.iloc[i]))
        
    count += 1
    
    
print('Giambullari: ', len(Giambullari), 'Borghini: ', len(Borghini), 'Bartoli: ', len(Bartoli))

Giambullari:  6 Borghini:  136 Bartoli:  73


### make single dataframes for Borghini, Bartoli and Giambullari

In [10]:
columns = ["Numero d'ordine", "Data", "Intestazione", "Segnatura", "Fonte", "Bibliografia"]

In [11]:
df_Giambullari = pd.DataFrame(Giambullari, columns = columns)
df_Borghini = pd.DataFrame(Borghini, columns = columns)
df_Bartoli = pd.DataFrame(Bartoli, columns = columns)

In [12]:
df_Borghini

,Numero d'ordine,Data,Intestazione,Segnatura,Fonte,Bibliografia
0,1051,26 12 1572,VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI ...,"ASA, AV, 14 (XLVIII), cc. 146-148.",Molto Magnifico messer Giorgio. Ho ricevuto la...,"Frey 1930, pp. 736-737."
1,512,8 06 1564,VINCENZO BORGHINI IN GLI ALBERI A GIORGIO VAS...,"ASA, AV, 14 (XLVIII), c. 41.",Magnifico Messer Giorgio. Io mi muoio di sonno...,"Frey 1930, p. 84."
2,912,22 09 1571,VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI ...,"ASA, AV, 14 (XLVIII), c. 101.",Molto Magnifico Signor Cavalier. Il Gabburra a...,"Frey 1930, pp. 607-608."
3,302,3 03 1560,VINCENZO BORGHINI IN POPPIANO A GIORGIO VASARI...,"ASA, AV, 14 (XLVIII), cc. 195-198.",Magnifico messer Giorgio. Come io son giunto i...,"Frey 1923, pp. 534-535."
4,617,6 05 1566,VINCENZO BORGHINI IN POPPIANO A GIORGIO VASARI...,"ASA, AV, 14 (XLVIII), cc. 219-220.",Molto Magnifico messer Giorgio. Trovandomi a P...,"Frey 1930, pp. 236-238."
...,...,...,...,...,...,...
131,602,1 01 1565,VINCENZO BORGHINI IN FIRENZE A GIOVANNI CACCIN...,"ASA, AV, 14 (XLVIII), c. 181.",Messer Giovanni. Se e’ non si pensa al canto a...,"Frey 1930, p. 217 (fine novembre-inizio dicemb..."
132,897,1 01 1571,VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI ...,"ASA, AV, 14 (XLVIII), c. 176.",Magnifico Signor mio. Non occorre dir cosa alc...,"Frey 1930, pp. 594-595 (fine luglio)."
133,751,15 05 1569,VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI ...,"ASPI, AOST, Suppliche ed Informazioni, Filza I...",Messer Giorgio onorando. Ho considerata questa...,"Frey 1930, pp. 423-424."
134,1091,8 05 1573,VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI ...,"ASA, AV, 14 (XLVIII), cc. 161-182.",Molto Magnifico messer Giorgio mio. Ho la vost...,"Frey 1930, pp. 783-784."


### save dataframes as csv files

In [58]:
data_path = 'data/results/letters/'

In [59]:
df_Giambullari.to_csv(data_path + 'Giambullari.csv', index = False, header=True)
df_Borghini.to_csv(data_path + 'Borghini.csv', index = False, header=True)
df_Bartoli.to_csv(data_path + 'Bartoli.csv', index = False, header=True)

### open csv files as dataframes

In [ ]:
data_path = 'data/results/letters/'
Gia = 'Giambullari.csv'
Bor = 'Borghini.csv'
Bar = 'Bartoli.csv'

df_Giambullari = pd.read_csv(data_path+Gia)
df_Borghini = pd.read_csv(data_path+Bor)
df_Bartoli = pd.read_csv(data_path+Bar)

### Helper Funcitons

In [17]:
# get rid of strange line break "\xa0"
# open a file and overwrite it

def fix(name):
    """ Fix a single .txt file
    """
    input = open(name, "rt")
    text = input.read()
    input.close()
    text = text.decode("utf-8")

    # Show if we get bad hits
    for c in text:
        if c == u"\xa0":
            print("Ufff")

    text = text.replace(u"\xa0", u" ")
    text = text.encode("utf-8")

    output = open(name, "wt")
    output.write(text)
    output.close()


# Process all .txt files in the
# current folder
for f in os.listdir(os.getcwd()):
    if f.endswith(".txt"):
        fix(f)

In [18]:
# obo.py
# get rid of tags without Beautiful Soup

def stripTags(pageContent):
    pageContents = str(pageContent)

    inside = 0
    text = ''

    for char in pageContents:
        if char == '<':
            inside = 1
        elif (inside == 1 and char == '>'):
            inside = 0
        elif inside == 1:
            continue
        else:
            text += char

    return text